# [A Diversity-Promoting Objective Function for Neural Conversation Models](https://arxiv.org/abs/1510.03055)

TLDR; seq2seqによる会話モデルはより安全でありきたりな文(「I don't know」など)を生成する傾向にある。著者達はこれは、機械翻訳と比べて会話文の生成タスクは多彩な出力文を扱う必要があり、seq2seqモデルがより安全な応答に高い確率を与えているためと推測している。著者達は目的関数を通常の尤度からMaximum Mutual Information(MMI; 最大相互情報量)に置き換えることで、生成文の入力文に対する依存関係のみでなく生成文を与えられた上での入力文の尤もらしさも考慮する枠組みを提案する。著者達はMMIモデルをTwitterとOpenSubtitlesのデータセットに適用し、通常のseq2seqに比べBLEUスコアと、また人の手による評価の双方で改善がみられたとしている。

---

#### Key Points

* Problem: ニューラルネットワークによるsequence-to-sequenceモデルによる会話の応答文生成は、入力に関係なく安全(safe)でありきたりの(commonplace)応答("I don't know"等)を生成する傾向にある
* Goal: sequence-to-sequenceモデルの目的関数に従来の尤度の代わりにMaximum Mutual Information(MMI; 最大相互情報量)を用い、より多彩で文法的に正しく整合性のとれた応答を生成する
* ニューラルネットワークによる会話モデルの生成する文のN-bestリスト中にはより意味のある又は明確な文も存在するが、これらはとてもランクが低い
* 入力を与えられた上で出力の尤度を最適化することでニューラルモデルは「安全」(safe)な応答に高い確率を割り当てていると考えられる
* 従来の目的関数は、機械翻訳のようなタスクにおいては一般的だが、本質的に多彩な文の生成タスクには適さないのかもしれない
* 目的関数にMaximum Mutual Information(MMI; 最大相互情報量)を用いることで、生成文の入力文に対する依存関係のみでなく生成文を与えられた上での入力文の尤もらしさも考慮するようにする
* MMIの目的関数: $\log { \frac{p(S|T)}{p(S)p(T)} } = \log p(T|S) - \lambda \log p(T)$
  * 出力を与えられたときの入力(p(S|T))と入力を与えられた時の出力(p(T|S))の間のトレードオフを表現しているとみなせる
* MMIの目的関数を直接学習する代わりに、学習時は最大尤度モデルを最適化し、テスト時のみMMI基準を用いる
  * $\lambda$の値をニューラルネットワークの最適化時に行おうとすると大量の時間を要するため
* MMI-antiLM: $\hat{T} = argmax_T \{ \log p(T|S) - \lambda \log p(T) \}$
* MMI-bidi: $\hat{T} = argmax_T \{ (1 - \lambda)\log p(T|S) + \lambda \log p(S|T) \}$
  * MMI-antiLMを式変形して得られる
* MMI-antiLMは文法的に正しくない出力を導きやすい
  * 出力単語の添字の増分に沿って減衰するペナルティを$p(S|T)$にかけて対処する
* MMI-bidiを直接計算するのは困難
  * まず目的関数の第1項$p(T|S)$(通常のseq2seq)でN-bestリストを生成し、第2項$p(S|T)$でこれをrerankする
* TwitterのtripleデータセットとOpenSubtitlesデータセットを使って実験した
* MMI-bidiはベースライン(seq2seq)に比べBLEUスコアで大きな改善が見られた
* 人の手による評価でもMMI-bidiはベースライン(seq2seq)に比べ良い結果となった


#### 参考

* Li, Jiwei, et al. "A diversity-promoting objective function for neural conversation models." arXiv preprint arXiv:1510.03055 (2015).

#### notes

$$
PMI = \log \frac {p(S,T)}{p(S)p(T)}
$$

MMIの目的関数はPMIをTについて最大化したもの

$$
\begin{eqnarray}
\hat{T} &=& argmax_T \{ \log \frac {p(S,T)}{p(S)p(T)} \} \\
&=& argmax_T \{ \log p(T|S) - \log p(T)\} \\
\end{eqnarray}
$$

通常の目的関数$argmax_T \log p(T|S)$に正則化項を加えたものとみなすことができる。正則化の度合いを制御するハイパーパラメーター$\lambda$を導入して

$$
\hat{T} = argmax_T \{ \log p(T|S) - \lambda \log p(T)\}
$$

これがMMI-antiLMの式。ここでPMIの式はベイズの定理を用いて以下の通り変形できる

$$
\begin{eqnarray}
\log \frac {p(S,T)}{p(S)p(T)} &=& \log \frac {p(S|T)}{p(S)} \\
\log p(T) &=& \log \frac {p(S,T)}{p(S)} \frac{p(S)}{p(S|T)} \\
&=& \log \frac {p(T|S)p(S)}{p(S|T)} \\
&=& \log p(T|S) + \log p(S) - \log p(S|T)
\end{eqnarray}
$$

これをMMIの目的関数に代入して

$$
\begin{eqnarray}
\hat{T} &=& argmax_T \{ \log P(T|S) - \lambda \log p(T) \} \\
&=& argmax_T \{ \log P(T|S) - \lambda (\log p(T|S) + \log p(S) - \log p(S|T)) \} \\
&=& argmax_T \{ (1 - \lambda) \log P(T|S) + \lambda \log p(S|T) - \lambda \log p(S)\} \\
&=& argmax_T \{ (1 - \lambda) \log P(T|S) + \lambda \log p(S|T)\} \\
\end{eqnarray}
$$

これがMMI-bidiの式。